In [1]:
import sys
import numpy as np
import pandas as pd
import seaborn as sns

sys.path.append('..')

import mim.extractors.ab_json as ab_json

In [2]:
specification = {
    "labels": {"target": "label-index+30d-ami+death-30d"},
    "index": {"json_train": "/home/sapfo/andersb/PycharmProjects/Expect/json_data/pontus_glukos/hbg+lund-train.json.gz"},
    "features":{
        "gender",
        "age",
        "bl-Glukos", "bl-TnT", "bl-Krea", "bl-Hb"
    },
}


In [3]:
extractor = ab_json.ABJSONExtractor(specification)

In [4]:
q = extractor.get_data()

{'bl-Glukos', 'gender', 'bl-TnT', 'bl-Krea', 'bl-Hb', 'age'}
['age', 'bl-Glukos', 'bl-Hb', 'bl-Krea', 'bl-TnT']
['gender']


In [5]:
q.shape

{'x': {'numeric': TensorShape([5]), 'categorical': TensorShape([2])},
 'y': TensorShape([])}

In [10]:
q["x"]["numeric"]

mim.extractors.extractor.Data

In [7]:
q["x"]["categorical"]

In [8]:
print(ab_json.NUMERICAL_FEATURES)

['age', 'bl-Glukos', 'bl-Krea', 'bl-TnT', 'bl-Hb']


In [14]:
q['y'].as_numpy.sum()

428

In [16]:
len(q['y'])

4761